## Byte-Pair Encoding tokenization

Byte-Pair Encoding (BPE) was initially developed as an algorithm to compress texts, and then used by OpenAI for tokenization when pretraining the GPT model. It’s used by a lot of Transformer models, including `GPT`, `GPT-2`, `RoBERTa`, `BART`, and `DeBERTa`.


### Training algorithm

BPE training starts by computing the unique set of words used in the corpus (after the normalization and pre-tokenization steps are completed), then building the vocabulary by taking all the symbols used to write those words. As a very simple example, let’s say our corpus uses these five words:

```
"hug", "pug", "pun", "bun", "hugs"
```

The base vocabulary will then be `["b", "g", "h", "n", "p", "s", "u"]`. For real-world cases, that base vocabulary will contain all the ASCII characters, at the very least, and probably some Unicode characters as well. If an example you are tokenizing uses a character that is not in the training corpus, that character will be converted to the unknown token. That’s one reason why lots of NLP models are very bad at analyzing content with emojis, for instance.

After getting this base vocabulary, we add new tokens until the desired vocabulary size is reached by learning merges, which are rules to merge two elements of the existing vocabulary together into a new one. So, at the beginning these merges will create tokens with two characters, and then, as training progresses, longer subwords.

At any step during the tokenizer training, the BPE algorithm will search for the most frequent pair of existing tokens (by “pair,” here we mean two consecutive tokens in a word). That most frequent pair is the one that will be merged, and we rinse and repeat for the next step.


Going back to our previous example, let’s assume the words had the following frequencies:
```
("hug", 10), ("pug", 5), ("pun", 12), ("bun", 4), ("hugs", 5)
```

meaning "hug" was present 10 times in the corpus, "pug" 5 times, "pun" 12 times, "bun" 4 times, and "hugs" 5 times. We start the training by splitting each word into characters (the ones that form our initial vocabulary) so we can see each word as a list of tokens:
```
("h" "u" "g", 10), ("p" "u" "g", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "u" "g" "s", 5)
```

Then we look at pairs. The pair ("h", "u") is present in the words "hug" and "hugs", so 15 times total in the corpus. It’s not the most frequent pair, though: that honor belongs to ("u", "g"), which is present in "hug", "pug", and "hugs", for a grand total of 20 times in the vocabulary.

Thus, the first merge rule learned by the tokenizer is ("u", "g") -> "ug", which means that "ug" will be added to the vocabulary, and the pair should be merged in all the words of the corpus. At the end of this stage, the vocabulary and corpus look like this:
```
Vocabulary: ["b", "g", "h", "n", "p", "s", "u", "ug"]
Corpus: ("h" "ug", 10), ("p" "ug", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "ug" "s", 5)
```

Now we have some pairs that result in a token longer than two characters: the pair `("h", "ug")`, for instance (present 15 times in the corpus). The most frequent pair at this stage is `("u", "n")`, however, present 16 times in the corpus, so the second merge rule learned is `("u", "n") -> "un"`. Adding that to the vocabulary and merging all existing occurrences leads us to:
```
Vocabulary: ["b", "g", "h", "n", "p", "s", "u", "ug", "un"]
Corpus: ("h" "ug", 10), ("p" "ug", 5), ("p" "un", 12), ("b" "un", 4), ("h" "ug" "s", 5)
```

Now the most frequent pair is ("h", "ug"), so we learn the merge rule `("h", "ug") -> "hug"`, which gives us our first three-letter token. After the merge, the corpus looks like this:
```
Vocabulary: ["b", "g", "h", "n", "p", "s", "u", "ug", "un", "hug"]
Corpus: ("hug", 10), ("p" "ug", 5), ("p" "un", 12), ("b" "un", 4), ("hug" "s", 5)
```

And we continue like this until we reach the desired vocabulary size.

## Tokenization algorithm

Tokenization follows the training process closely, in the sense that new inputs are tokenized by applying the following steps:

1. Normalization
   
2. Pre-tokenization
   
3. Splitting the words into individual characters
   
4. Applying the merge rules learned in order on those splits


Let’s take the example we used during training, with the three merge rules learned:
```
("u", "g") -> "ug"
("u", "n") -> "un"
("h", "ug") -> "hug"
```

The word `"bug"` will be tokenized as `["b", "ug"]`. `"mug"`, however, will be tokenized as `["[UNK]", "ug"]` since the letter `"m"` was not in the base vocabulary. Likewise, the word `"thug"` will be tokenized as `["[UNK]", "hug"]`: the letter `"t"` is not in the base vocabulary, and applying the merge rules results first in `"u"` and `"g"` being merged and then `"h"` and `"ug"` being merged.

## Implementing BPE

Now let’s take a look at an implementation of the BPE algorithm. This won’t be an optimized version you can actually use on a big corpus; we just want to show you the code so you can understand the algorithm a little bit better.

First we need a corpus, so let’s create a simple one with a few sentences:


In [1]:
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

Next, we need to pre-tokenize that corpus into words. Since we are replicating a BPE tokenizer (like GPT-2), we will use the gpt2 tokenizer for the pre-tokenization:

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

Then we compute the frequencies of each word in the corpus as we do the pre-tokenization:

In [9]:
from collections import defaultdict

word_freqs = defaultdict(int)

for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

print(word_freqs)

defaultdict(<class 'int'>, {'This': 3, 'Ġis': 2, 'Ġthe': 1, 'ĠHugging': 1, 'ĠFace': 1, 'ĠCourse': 1, '.': 4, 'Ġchapter': 1, 'Ġabout': 1, 'Ġtokenization': 1, 'Ġsection': 1, 'Ġshows': 1, 'Ġseveral': 1, 'Ġtokenizer': 1, 'Ġalgorithms': 1, 'Hopefully': 1, ',': 1, 'Ġyou': 1, 'Ġwill': 1, 'Ġbe': 1, 'Ġable': 1, 'Ġto': 1, 'Ġunderstand': 1, 'Ġhow': 1, 'Ġthey': 1, 'Ġare': 1, 'Ġtrained': 1, 'Ġand': 1, 'Ġgenerate': 1, 'Ġtokens': 1})


The next step is to compute the base vocabulary, formed by all the characters used in the corpus:


In [12]:
word_freqs.keys(), word_freqs.values()

(dict_keys(['This', 'Ġis', 'Ġthe', 'ĠHugging', 'ĠFace', 'ĠCourse', '.', 'Ġchapter', 'Ġabout', 'Ġtokenization', 'Ġsection', 'Ġshows', 'Ġseveral', 'Ġtokenizer', 'Ġalgorithms', 'Hopefully', ',', 'Ġyou', 'Ġwill', 'Ġbe', 'Ġable', 'Ġto', 'Ġunderstand', 'Ġhow', 'Ġthey', 'Ġare', 'Ġtrained', 'Ġand', 'Ġgenerate', 'Ġtokens']),
 dict_values([3, 2, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]))

In [13]:
for word in word_freqs.keys():
    print(word)

This
Ġis
Ġthe
ĠHugging
ĠFace
ĠCourse
.
Ġchapter
Ġabout
Ġtokenization
Ġsection
Ġshows
Ġseveral
Ġtokenizer
Ġalgorithms
Hopefully
,
Ġyou
Ġwill
Ġbe
Ġable
Ġto
Ġunderstand
Ġhow
Ġthey
Ġare
Ġtrained
Ġand
Ġgenerate
Ġtokens


In [10]:
alphabet = []

for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()

print(alphabet)

[',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', 'Ġ']


We also add the special tokens used by the model at the beginning of that vocabulary. In the case of GPT-2, the only special token is `"<|endoftext|>"`:

In [15]:
vocab = ["<|endoftext|>"] + alphabet.copy()

In [16]:
vocab

['<|endoftext|>',
 ',',
 '.',
 'C',
 'F',
 'H',
 'T',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'y',
 'z',
 'Ġ']

We now need to split each word into individual characters, to be able to start training:

In [17]:
splits = {word: [c for c in word] for word in word_freqs.keys()}

In [18]:
splits

{'This': ['T', 'h', 'i', 's'],
 'Ġis': ['Ġ', 'i', 's'],
 'Ġthe': ['Ġ', 't', 'h', 'e'],
 'ĠHugging': ['Ġ', 'H', 'u', 'g', 'g', 'i', 'n', 'g'],
 'ĠFace': ['Ġ', 'F', 'a', 'c', 'e'],
 'ĠCourse': ['Ġ', 'C', 'o', 'u', 'r', 's', 'e'],
 '.': ['.'],
 'Ġchapter': ['Ġ', 'c', 'h', 'a', 'p', 't', 'e', 'r'],
 'Ġabout': ['Ġ', 'a', 'b', 'o', 'u', 't'],
 'Ġtokenization': ['Ġ',
  't',
  'o',
  'k',
  'e',
  'n',
  'i',
  'z',
  'a',
  't',
  'i',
  'o',
  'n'],
 'Ġsection': ['Ġ', 's', 'e', 'c', 't', 'i', 'o', 'n'],
 'Ġshows': ['Ġ', 's', 'h', 'o', 'w', 's'],
 'Ġseveral': ['Ġ', 's', 'e', 'v', 'e', 'r', 'a', 'l'],
 'Ġtokenizer': ['Ġ', 't', 'o', 'k', 'e', 'n', 'i', 'z', 'e', 'r'],
 'Ġalgorithms': ['Ġ', 'a', 'l', 'g', 'o', 'r', 'i', 't', 'h', 'm', 's'],
 'Hopefully': ['H', 'o', 'p', 'e', 'f', 'u', 'l', 'l', 'y'],
 ',': [','],
 'Ġyou': ['Ġ', 'y', 'o', 'u'],
 'Ġwill': ['Ġ', 'w', 'i', 'l', 'l'],
 'Ġbe': ['Ġ', 'b', 'e'],
 'Ġable': ['Ġ', 'a', 'b', 'l', 'e'],
 'Ġto': ['Ġ', 't', 'o'],
 'Ġunderstand': ['Ġ', 'u', 'n'

Now that we are ready for training, let’s write a function that computes the frequency of each pair. We’ll need to use this at each step of the training:

In [24]:
word_freqs.items()

dict_items([('This', 3), ('Ġis', 2), ('Ġthe', 1), ('ĠHugging', 1), ('ĠFace', 1), ('ĠCourse', 1), ('.', 4), ('Ġchapter', 1), ('Ġabout', 1), ('Ġtokenization', 1), ('Ġsection', 1), ('Ġshows', 1), ('Ġseveral', 1), ('Ġtokenizer', 1), ('Ġalgorithms', 1), ('Hopefully', 1), (',', 1), ('Ġyou', 1), ('Ġwill', 1), ('Ġbe', 1), ('Ġable', 1), ('Ġto', 1), ('Ġunderstand', 1), ('Ġhow', 1), ('Ġthey', 1), ('Ġare', 1), ('Ġtrained', 1), ('Ġand', 1), ('Ġgenerate', 1), ('Ġtokens', 1)])

In [67]:
pair_freqs = defaultdict(int)
for word, freq in word_freqs.items():
    split = splits[word]
    print(freq)
    print(split)
    print(range(len(split)))

3
['T', 'h', 'i', 's']
range(0, 4)
2
['Ġ', 'i', 's']
range(0, 3)
1
['Ġ', 't', 'h', 'e']
range(0, 4)
1
['Ġ', 'H', 'u', 'g', 'g', 'i', 'n', 'g']
range(0, 8)
1
['Ġ', 'F', 'a', 'c', 'e']
range(0, 5)
1
['Ġ', 'C', 'o', 'u', 'r', 's', 'e']
range(0, 7)
4
['.']
range(0, 1)
1
['Ġ', 'c', 'h', 'a', 'p', 't', 'e', 'r']
range(0, 8)
1
['Ġ', 'a', 'b', 'o', 'u', 't']
range(0, 6)
1
['Ġ', 't', 'o', 'k', 'e', 'n', 'i', 'z', 'a', 't', 'i', 'o', 'n']
range(0, 13)
1
['Ġ', 's', 'e', 'c', 't', 'i', 'o', 'n']
range(0, 8)
1
['Ġ', 's', 'h', 'o', 'w', 's']
range(0, 6)
1
['Ġ', 's', 'e', 'v', 'e', 'r', 'a', 'l']
range(0, 8)
1
['Ġ', 't', 'o', 'k', 'e', 'n', 'i', 'z', 'e', 'r']
range(0, 10)
1
['Ġ', 'a', 'l', 'g', 'o', 'r', 'i', 't', 'h', 'm', 's']
range(0, 11)
1
['H', 'o', 'p', 'e', 'f', 'u', 'l', 'l', 'y']
range(0, 9)
1
[',']
range(0, 1)
1
['Ġ', 'y', 'o', 'u']
range(0, 4)
1
['Ġ', 'w', 'i', 'l', 'l']
range(0, 5)
1
['Ġ', 'b', 'e']
range(0, 3)
1
['Ġ', 'a', 'b', 'l', 'e']
range(0, 5)
1
['Ġ', 't', 'o']
range(0, 3)
1
['Ġ',

In [65]:
for i in range(len(split) - 1):
    pair = (split[i], split[i + 1])
    pair_freqs[pair] += freq

In [66]:
pair_freqs

defaultdict(int,
            {('Ġ', 't'): 1,
             ('t', 'o'): 1,
             ('o', 'k'): 1,
             ('k', 'e'): 1,
             ('e', 'n'): 1,
             ('n', 's'): 1})

In [68]:
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

Let’s have a look at a part of this dictionary after the initial splits:

In [69]:
pair_freqs = compute_pair_freqs(splits)

for i, key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")
    if i >= 5:
        break

('T', 'h'): 3
('h', 'i'): 3
('i', 's'): 5
('Ġ', 'i'): 2
('Ġ', 't'): 7
('t', 'h'): 3


Now, finding the most frequent pair only takes a quick loop:

In [99]:
for pair, freq in pair_freqs.items():
    print(pair, freq)

('T', 'h') 3
('h', 'i') 3
('i', 's') 5
('Ġ', 'i') 2
('Ġ', 't') 7
('t', 'h') 3
('h', 'e') 2
('Ġ', 'H') 1
('H', 'u') 1
('u', 'g') 1
('g', 'g') 1
('g', 'i') 1
('i', 'n') 2
('n', 'g') 1
('Ġ', 'F') 1
('F', 'a') 1
('a', 'c') 1
('c', 'e') 1
('Ġ', 'C') 1
('C', 'o') 1
('o', 'u') 3
('u', 'r') 1
('r', 's') 2
('s', 'e') 3
('Ġ', 'c') 1
('c', 'h') 1
('h', 'a') 1
('a', 'p') 1
('p', 't') 1
('t', 'e') 2
('e', 'r') 5
('Ġ', 'a') 5
('a', 'b') 2
('b', 'o') 1
('u', 't') 1
('t', 'o') 4
('o', 'k') 3
('k', 'e') 3
('e', 'n') 4
('n', 'i') 2
('i', 'z') 2
('z', 'a') 1
('a', 't') 2
('t', 'i') 2
('i', 'o') 2
('o', 'n') 2
('Ġ', 's') 3
('e', 'c') 1
('c', 't') 1
('s', 'h') 1
('h', 'o') 2
('o', 'w') 2
('w', 's') 1
('e', 'v') 1
('v', 'e') 1
('r', 'a') 3
('a', 'l') 2
('z', 'e') 1
('l', 'g') 1
('g', 'o') 1
('o', 'r') 1
('r', 'i') 1
('i', 't') 1
('h', 'm') 1
('m', 's') 1
('H', 'o') 1
('o', 'p') 1
('p', 'e') 1
('e', 'f') 1
('f', 'u') 1
('u', 'l') 1
('l', 'l') 2
('l', 'y') 1
('Ġ', 'y') 1
('y', 'o') 1
('Ġ', 'w') 1
('w', 'i') 1

In [100]:
best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)      

('Ġ', 't') 7


So the first merge to learn is `('Ġ', 't') -> 'Ġt'`, and we add 'Ġt' to the vocabulary:

In [101]:
merges = {("Ġ", "t"): "Ġt"}
vocab.append("Ġt")

In [102]:
vocab

['<|endoftext|>',
 ',',
 '.',
 'C',
 'F',
 'H',
 'T',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'y',
 'z',
 'Ġ',
 'Ġt']

To continue, we need to apply that merge in our splits dictionary. Let’s write another function for this:

In [103]:
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue
        
        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

And we can have a look at the result of the first merge:

In [122]:
splits = merge_pair("G", "t", splits)
print(splits["Ġtrained"])

['Ġ', 't', 'r', 'a', 'i', 'n', 'e', 'd']


Now we have everything we need to loop until we have learned all the merges we want. Let’s aim for a vocab size of 50:

In [125]:
vocab_size = 50

while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits) 
    best_pair = ""
    max_freq = None
    
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits = merge_pair(*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

As a result, we’ve learned 19 merge rules (the initial vocabulary had a size of 31 — 30 characters in the alphabet, plus the special token):

In [127]:
print(merges)

{('Ġ', 't'): 'Ġt', ('i', 's'): 'is', ('e', 'r'): 'er', ('Ġ', 'a'): 'Ġa', ('Ġt', 'o'): 'Ġto', ('e', 'n'): 'en', ('T', 'h'): 'Th', ('Th', 'is'): 'This', ('o', 'u'): 'ou', ('s', 'e'): 'se', ('Ġto', 'k'): 'Ġtok', ('Ġtok', 'en'): 'Ġtoken', ('n', 'd'): 'nd', ('Ġ', 'is'): 'Ġis', ('Ġt', 'h'): 'Ġth', ('Ġth', 'e'): 'Ġthe', ('i', 'n'): 'in', ('Ġa', 'b'): 'Ġab', ('Ġtoken', 'i'): 'Ġtokeni'}


And the vocabulary is composed of the special token, the initial alphabet, and all the results of the merges:

In [128]:
print(vocab)

['<|endoftext|>', ',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', 'Ġ', 'Ġt', 'is', 'er', 'Ġa', 'Ġto', 'en', 'Th', 'This', 'ou', 'se', 'Ġtok', 'Ġtoken', 'nd', 'Ġis', 'Ġth', 'Ġthe', 'in', 'Ġab', 'Ġtokeni']


To tokenize a new text, we pre-tokenize it, split it, then apply all the merge rules learned:

In [129]:
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    splits = [[l for l in word] for word in pre_tokenized_text]
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split
    
    return sum(splits, [])

We can try this on any text composed of characters in the alphabet:

In [130]:
tokenize("This is not a token.")

['This', 'Ġis', 'Ġ', 'n', 'o', 't', 'Ġa', 'Ġtoken', '.']